# improts

In [1]:
import pandas as pd
from notion_client import Client


# function definition

In [2]:
def get_text_content(prop_value, prop_type='rich_text'):
    if not prop_value:
        return ''
    try:
        if prop_type == 'date':
            return prop_value.get('date', {}).get('start', '')
        if prop_type == 'rich_text':
            texts = prop_value.get('rich_text', [])
            if not texts:
                return ''
            first_text = texts[0].get('text', {})
            return first_text.get('content', '')
        elif prop_type == 'title':
            titles = prop_value.get('title', [])
            if not titles:
                return ''
            first_title = titles[0].get('text', {})
            return first_title.get('content', '')
        elif prop_type == 'email':
            return prop_value.get('email', '')
    except (AttributeError, IndexError, KeyError):
        return ''
    return ''

# notions setup

In [3]:
# Initialize Notion client
NOTION_TOKEN = "ntn_S6159294934albrajfceBHL4szrrrMllKAcFNUGM62v7JI"
DATABASE_ID = "18316f1f61d680a2921bd08b8c62f895"
notion = Client(auth=NOTION_TOKEN)

In [ ]:
response = notion.databases.query(database_id=DATABASE_ID)
print(response['results'][0]['id'])
print(response['results'][1]['id'])
print(response['results'][2]['id'])
print(response['results'][3]['id'])

19716f1f-61d6-805f-9a30-f1275f120bc4
18516f1f-61d6-8056-acb8-d896832942bd
18516f1f-61d6-80cc-8477-d40b7aa4ccd1
18316f1f-61d6-80b5-b8d8-f497ca2ce124


# Pull database

In [ ]:
# create empty data arrey
data = []

# call notion API 
response = notion.databases.query(database_id=DATABASE_ID)

# iterate through each page and collect data
for page in response['results']:
    properties = page.get('properties', {})
    # Extract specific fields from the page properties
    row = {
        'notion raw id': page['id'],
        'customer name': page['properties']['First name']['title'][0]['plain_text'],
        'customer last name': page['properties']['Last name']['rich_text'][0]['plain_text'],
        'Phone': page['properties']['Main contact phone']['phone_number'],
        'Email': page['properties']['Main contact Email']['email'],
        'home address': page['properties']['Property Street']['rich_text'][0]['plain_text'],
        'Next Follow Up Date': page['properties']['Next Follow Up Date']['date']['start'] if page['properties']['Next Follow Up Date']['date'] else '',
        'Claim': page['properties']['Status of Claim']['status']['name'],
        'Lead': page['properties']['Status of lead']['status']['name'],   
        'call id' : page['properties']['Last call ID']['rich_text'][0]['plain_text'] if page['properties']['Last call ID']['rich_text'] else '',
        'First_call_transcription': page['properties']['cold call summery']['rich_text'][0]['plain_text'] if page['properties']['cold call summery']['rich_text'] else ''
    }
    # append data to the data arrey
    data.append(row)
    
# convert data to a dataframe   
df = pd.DataFrame(data)
df

,notion raw id,customer name,customer last name,Phone,Email,home address,Next Follow Up Date,Claim,Lead,call id,First_call_transcription
0,19716f1f-61d6-805f-9a30-f1275f120bc4,GEORGE,HOYLE,+972545842444,info@trueclaim.org,144 RIVER WATCH TER,2025-02-13T12:00:00.000+00:00,Not approved by client,Interested,,
1,18516f1f-61d6-8056-acb8-d896832942bd,test,test\t,131243,sssss@gmail.com,test1,2025-02-06T12:00:00.000+00:00,Not approved by client,Interested,123aa,test test test
2,18516f1f-61d6-80cc-8477-d40b7aa4ccd1,test,test\t,131243,sda@askld.com,test1,2025-02-06T06:00:00.000+02:00,Not approved by client,Interested,,
3,18316f1f-61d6-80b5-b8d8-f497ca2ce124,test,test\t,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Done,Fund recived,,
4,18316f1f-61d6-80ef-9328-f5a1afe5e9c3,test,test\t,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Sent to Attorney,Client singed documents,,
5,18316f1f-61d6-80bd-964b-d745ce018dfd,test,test\t,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Sent to Attorney,Interested,,
6,18316f1f-61d6-8088-975e-e871e191f0e0,test,test\t,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Approved and waiting for filled documents,Documents sent for signiture,1235aa,
7,18316f1f-61d6-80e5-b4d0-fd4b638c1f54,test,test\t,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Not approved by client,Interested,,


# Pull database only for cold call

In [6]:
cold_call_df = df[df['Lead'].isin(['New', 'Did not answered'])]
cold_call_df

,customer name,Phone,Email,home address,Next Follow Up Date,Claim,Lead,call id,First_call_transcription
0,test,,sda@askld.com,test1,,Not approved by client,New,123aa,
1,test,,sda@askld.com,test1,2025-02-06T00:00:00.000+02:00,Not approved by client,Did not answered,,


# Pull database only for closer

In [ ]:
mask = df['call id'] == "123aa"
df.loc[mask, 'First_call_transcription'].text

In [ ]:
closer_df = df[df['Lead'].isin(['Interested'])]
closer_df

# update database test

In [5]:
# Find the row with call id "123aa"
mask = df['call id'] == "123aa"

# Update the "First_call_transcription" column for the matched row
df.loc[mask, 'First_call_transcription'] = """
user: Hello? \n assistant: Hi
is this John? \n user: Yes. It is. \n assistant: Hi John
my name is Cristin and I'm calling from TrueClaim Services. I've spent the last several days trying to find you. We specialize in auditing public records to search for unclaimed funds held by government agencies and owed to individuals. I believe I've found some funds that might belong to you. Can you confirm if you used to live at 123 Main St before? \n user: Yes. It is. \n assistant: John
it appears that the local courts are holding onto some funds that belong to you
likely from a past foreclosure. These funds are at risk of being turned over to the government if not claimed in a timely manner. We'd like to help you recover them. Our service takes care of everything
from preparing and submitting the necessary paperwork to dealing with the bureaucracy
all at no upfront cost or risk to you. We only get paid if you receive your money back. Would you like to know more about the process and potentially recovering your funds? \n user: Yeah. Of course. \n assistant: I'm going to send over some information documents for you to review at your convenience. Additionally
I'm setting up a call with one of our senior representatives who will guide you through the process and answer any questions you may have. When would you prefer us to reach out to you to schedule this call? \n user: It's today. \n user: When \n assistant: I can arrange for the senior representative to call you today. Would you be available to take the call within the next hour or would you prefer a specific time that suits you better? \n user: Yeah. Next hour. \n assistant: To ensure you receive all necessary documents and updates on your request
could you please share your email address with me? \n user: Yeah. Sure. And email is john@gmail.com. \n assistant: Just to confirm
your email is john@gmail.com
correct? \n user: Yes. It is. \n assistant: Thank you so much for taking the time to chat with me today
John. \n 
"""

# Update the Notion database 
for index, row in df[mask].iterrows():
    page_id = response['results'][index]['id']
    notion.pages.update(
        page_id=page_id,
        properties={
            'cold call transcription': {
                'rich_text': [{'text': {'content': row['First_call_transcription']}}]
            }
        }
    )

print("Notion database updated successfully.")


Notion database updated successfully.


In [82]:
call_info_dict = {}

# collect costumer email
call_info_dict["Main contact Email"] = 'sssss@gmail.com'
# collect follow up date from transcription
call_info_dict["Next Follow Up Date"]  = '2025-02-06 12:00:00'
# collect lead status
call_info_dict["Status of lead"] = 'Interested'
# collect cold call transcription
call_info_dict["cold call transcription"] = "test test test"  

# update notion table
notion.pages.update(
    page_id="18516f1f-61d6-8056-acb8-d896832942bd",
    properties={
        'cold call transcription': {
            'rich_text': [
                {
                    'text': {
                        'content': call_info_dict["cold call transcription"]
                    }
                }
            ]
        },
        'Status of lead': {
            'status': {
                'name': call_info_dict["Status of lead"]
            }
        },
        'Main contact Email': {
            'email': call_info_dict["Main contact Email"]
        },
        'Next Follow Up Date': {
            'date': {
                'start': call_info_dict["Next Follow Up Date"]
            }
        }
    }
)
        

{'object': 'page',
 'id': '18516f1f-61d6-8056-acb8-d896832942bd',
 'created_time': '2025-01-24T19:29:00.000Z',
 'last_edited_time': '2025-02-10T22:21:00.000Z',
 'created_by': {'object': 'user',
  'id': 'f5725b83-56c8-4578-8dd4-a409ce5091f7'},
 'last_edited_by': {'object': 'user',
  'id': '6cb362cc-b7c9-41c6-8a6a-d16fc5e0d7b7'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'database_id',
  'database_id': '18316f1f-61d6-80a2-921b-d08b8c62f895'},
 'archived': False,
 'in_trash': False,
 'properties': {'state': {'id': '%3ALr%7B',
   'type': 'rich_text',
   'rich_text': [{'type': 'text',
     'text': {'content': 'test1', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'test1',
     'href': None}]},
  'Main contact Email': {'id': '%3C%3A%7Bw',
   'type': 'email',
   'email': 'sssss@gmail.com'},
  'City': {'id': '%3DcWJ',
   'type': 'rich_t